In [ ]:
import os
import numpy as np
import torch
from tqdm.notebook import tqdm

torch.set_grad_enabled(False)
DEVICE = "cuda:0"


def normalize(image):
    image = image / 255
    image = (image - 0.5) / 0.5
    return image

def denormalize(image):
    image = (image * 0.5) + 0.5
    image = image * 255
    image = torch.clip(image, min=0, max=255.0)
    return image

# 1. Load Model

### 1.1. Load Generator from source code 

In [ ]:
from models import Generator

CKPT_PATH = "./src/ckpt_E:213.ckpt.pth"

assert os.path.exists(CKPT_PATH), f"{CKPT_PATH} Not Found"


class args:
    image_channels = 3
    g_dim = 32


model = Generator(args())
ckpt = torch.load(CKPT_PATH, map_location="cpu")
model.load_state_dict(ckpt["gen"])
model = model.to(DEVICE).eval()


### 1.2. Load Generator from torchscript

In [ ]:
TORCHSCRIPT_PATH = './src/AnimeGAN.pt.zip'
assert os.path.exists(TORCHSCRIPT_PATH), f"{TORCHSCRIPT_PATH} Not Found"

model = torch.jit.load(TORCHSCRIPT_PATH)
model = model.to(DEVICE).eval()

# 2. Photo Convert

In [ ]:
import cv2
from PIL import Image

TARGET_PHOTO_FILE_PATH = "./src/temp.png"
! curl -o {TARGET_PHOTO_FILE_PATH} "https://scontent-ssn1-1.xx.fbcdn.net/v/t31.18172-8/22382411_10159396144300134_2436048902354400141_o.jpg?_nc_cat=104&ccb=1-5&_nc_sid=730e14&_nc_ohc=MvqDCYs75IwAX-F5ALB&_nc_ht=scontent-ssn1-1.xx&oh=00_AT9MswhisOTffXl0suHiS3_ue_uyx9RRjiJEzBck9mBmQQ&oe=626724FD"

assert os.path.exists(
    TARGET_PHOTO_FILE_PATH
), f"{TARGET_PHOTO_FILE_PATH} not Found"

image = cv2.imread(TARGET_PHOTO_FILE_PATH)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
Image.fromarray(image).show()
image = np.expand_dims(image, 0)
image = np.transpose(image, [0, 3, 2, 1])
image = image.astype(np.float32)
image = normalize(torch.from_numpy(image))
image = image.to(DEVICE)
with torch.cuda.amp.autocast():
    image = model(image)

image = denormalize(image).cpu().numpy()
image = np.transpose(image, [0, 3, 2, 1])
image = image.astype(np.uint8)
Image.fromarray(image[0]).show()


# 3. Video Convert

### 3.1. Video file download

In [ ]:
import tempfile
TARGET_VIDEO_FILE_PATH = "./src/video/sample.mp4"
SAVE_VIDEO_DIR = "./src/video/"

# download video file from youtube
! yt-dlp -f 299+140 https://www.youtube.com/watch?v=KoMw2Qa5bQs  -o {TARGET_VIDEO_FILE_PATH}

assert os.path.exists(
    TARGET_VIDEO_FILE_PATH
), f"{TARGET_VIDEO_FILE_PATH} Not Found"

os.makedirs(SAVE_VIDEO_DIR, exist_ok=True)

basename = os.path.basename(TARGET_VIDEO_FILE_PATH)
basename = os.path.splitext(basename)[0]

SAVE_VIDEO_FILE_PATH = os.path.join(SAVE_VIDEO_DIR, basename + "_anime_wo_audio.mp4")
VIDEO_WITH_AUDIO_FILE = os.path.join(
    SAVE_VIDEO_DIR, basename + "_anime.mp4"
)

temp = tempfile.NamedTemporaryFile(suffix=".wav")
AUDIO_PATH = temp.name

### 3.2. Video file setting

In [ ]:
from moviepy.editor import VideoFileClip
import moviepy.video.io.ffmpeg_writer as ffmpeg_writer

video_clip = VideoFileClip(TARGET_VIDEO_FILE_PATH)
audio_clip = video_clip.audio
audio_clip.write_audiofile(AUDIO_PATH)
total_frames = int(video_clip.duration * video_clip.fps)

video_writer = ffmpeg_writer.FFMPEG_VideoWriter(
    SAVE_VIDEO_FILE_PATH,
    video_clip.size,
    video_clip.fps,
    codec="libx264",
    preset="medium",
    bitrate="5500k",
    threads=8,
    ffmpeg_params=None,
)


### 3.3. Convert Video

In [ ]:
# adjust BATCH_SIZE to your GPU vRAM
BATCH_SIZE = 6
buf = []

assert os.path.exists(AUDIO_PATH), f"{AUDIO_PATH} Not Found"

for idx, frame in enumerate(tqdm(video_clip.iter_frames(), total=total_frames)):
    buf.append(frame)

    if len(buf) < BATCH_SIZE and idx < total_frames - 1:
        continue

    images = [np.expand_dims(image, 0) for image in buf]
    images = np.concatenate(images, 0)
    images = np.transpose(images, [0, 3, 2, 1])
    images = images.astype(np.float32)

    images = normalize(torch.from_numpy(images))
    images = images.to(DEVICE)

    # inference
    with torch.cuda.amp.autocast():
        processed_image = model(images)

    processed_image = denormalize(processed_image)
    processed_image = processed_image.cpu().numpy()
    processed_image = np.transpose(processed_image, [0, 3, 2, 1])
    processed_image = processed_image.astype(np.uint8)
    frames = [frame for frame in processed_image]
    for frame in frames:
        video_writer.write_frame(frame)
    buf.clear()


In [ ]:
# mix audio
! ffmpeg -i {SAVE_VIDEO_FILE_PATH} -i {AUDIO_PATH} {VIDEO_WITH_AUDIO_FILE}

In [ ]:
TARGET_VIDEO = "src/video/sample.mp4"
GIF_PATH = 'src/gif/sample_origin.gif'

TARGET_ANIME_VIDEO = "src/video/sample_anime.mp4"
ANIME_GIF_PATH = "src/gif/sample_anime.gif"

! ffmpeg -i {TARGET_VIDEO} -vf scale=320:-1 -r 10 -f image2pipe -vcodec ppm - | convert -delay 10 -loop 0 - {GIF_PATH}
! ffmpeg -i {TARGET_ANIME_VIDEO} -vf scale=720:-1 -r 10 -f image2pipe -vcodec ppm - | convert -delay 10 -loop 0 - {ANIME_GIF_PATH}